In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [ ]:
nltk.download()

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')

In [4]:
alice = gutenberg.raw('carroll-alice.txt')

In [5]:
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)

In [6]:
alice = re.sub(r'CHAPTER .*', '', alice)

In [7]:
alice = text_cleaner(alice)

In [8]:
persuasion = text_cleaner(persuasion)

In [9]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [10]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,"((, when, she, thought, it, over, afterwards, ...",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [11]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [12]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,rest,gratify,indignant,probably,heir,lovely,season,revival,consent,chop,...,zigzag,dignity,learn,somebody,talent,old,ere,mistress,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [32]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.993392070485

Test set score: 0.918502202643


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2724, 3023) (2724,)
Training set score: 0.963656387665

Test set score: 0.935572687225


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [15]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.921439060206

Test set score: 0.906938325991


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [17]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [18]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [19]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [21]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [64]:
# Challenge 0
from sklearn.svm import SVC

# Trying out SVC
# Make a model using SVC here
svc = SVC()
svc.fit(X_train, y_train)

# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.734581497797


col_0,Austen,Carroll
text_source,,
Austen,1333,0
Carroll,482,1


In [39]:
# The SVC model did not predict any titles by 'Carroll'

In [46]:
# making more features that take advantage of the spaCy information 
# (include grammar, phrases, POS, etc)
# Will include pos to see how it helps 
def parts_of_speech(text):

    pos = [token.pos_
          for token in text
          if not token.is_punct
          and not token.is_stop]
    return [item for item in pos]

def pos_features(sentences, common_pos):
    d2 = pd.DataFrame(columns=common_pos)
    d2['text_sentence'] = sentences[0]
    d2['text_source'] = sentences[1]
    d2.loc[:, common_pos] = 0
    
    for i, sentence in enumerate(d2['text_sentence']):
        parts = [token.pos_
                for token in sentence
                if (
                    not token.is_punct
                    and not token.is_stop
                    and token.pos_ in common_pos
                )]
        for part in parts:
            d2.loc[i, part] += 1
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return d2

# Set up the groups.
alicepos = parts_of_speech(alice_doc)
persuasionpos = parts_of_speech(persuasion_doc)

# Combine groups to create a set of parts of speech.
common_pos = set(alicepos + persuasionpos)

In [47]:
# Create our data frame with features. This can take a while to run.
pos_counts = pos_features(sentences, common_pos)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,NOUN,ADJ,INTJ,PUNCT,ADP,PROPN,VERB,DET,NUM,PART,ADV,PRON,X,text_sentence,text_source
0,10,1,0,0,0,2,6,0,0,0,1,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,8,5,0,0,0,2,6,0,0,0,2,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,1,1,2,0,0,2,2,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,1,2,0,0,0,1,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,14,2,0,0,0,4,13,0,0,0,2,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [48]:
# Merge the two dataframes
frames = [word_counts, pos_counts]

combined = pd.concat(frames, axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
combined.head()

,rest,gratify,indignant,probably,heir,lovely,season,revival,consent,chop,...,PROPN,VERB,DET,NUM,PART,ADV,PRON,X,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,2,6,0,0,0,1,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,2,6,0,0,0,2,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,4,13,0,0,0,2,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [57]:
print(np.where(combined.columns == 'text_source'))

(array([3024, 3039], dtype=int64),)


In [60]:
# Since there are two columns for 'text_source,' drop one of them
combined = combined.groupby(level=0, axis=1).last()

In [61]:
print(np.where(combined.columns == 'text_source'))

(array([2708], dtype=int64),)


In [62]:
Y = combined['text_source']
X = np.array(combined.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [63]:
Y = combined['text_source']
X = np.array(combined.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.963656387665

Test set score: 0.93281938326


Adding parts of speech did not improve accuracy (previous was .935)

In [65]:
# Challenge 0
from sklearn.svm import SVC

# Trying out SVC
# Make a model using SVC here
svc = SVC()
svc.fit(X_train, y_train)

# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.734581497797


col_0,Austen,Carroll
text_source,,
Austen,1333,0
Carroll,482,1


making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), 

SVC did not perform better with the new features

In [87]:
mean_pos = combined[['NOUN','ADJ','INTJ','PUNCT','ADP','PROPN','VERB','DET','NUM','PART','ADV','PRON','X','text_source']].groupby(['text_source']).mean()
                 
print(mean_pos)

                 NOUN       ADJ      INTJ     PUNCT       ADP     PROPN  \
text_source                                                               
Austen       3.433204  1.343945  0.043142  0.008628  0.044332  1.487355   
Carroll      2.910093  0.953350  0.081425  0.000000  0.077184  1.103478   

                 VERB       DET       NUM      PART       ADV      PRON  \
text_source                                                               
Austen       2.607260  0.000595  0.019935  0.141327  0.477536  0.001488   
Carroll      3.430874  0.002545  0.007634  0.044105  0.639525  0.011026   

                    X  
text_source            
Austen       0.000000  
Carroll      0.000848  


In [98]:
# Drop features where the means of the pos are similar (based on standard deviation)
std_pos = combined[['NOUN','ADJ','INTJ','PUNCT','ADP','PROPN','VERB','DET','NUM','PART','ADV','PRON','X','text_source']].groupby(['text_source']).std()

print(std_pos)

                 NOUN       ADJ      INTJ     PUNCT       ADP     PROPN  \
text_source                                                               
Austen       3.756552  1.726578  0.229352  0.115405  0.214361  2.098874   
Carroll      3.504844  1.368588  0.300231  0.000000  0.288395  1.346370   

                 VERB      DET       NUM      PART       ADV      PRON  \
text_source                                                              
Austen       2.551236  0.02439  0.268186  0.411107  0.808540  0.038547   
Carroll      3.021867  0.05040  0.096330  0.221330  0.974983  0.104470   

                    X  
text_source            
Austen       0.000000  
Carroll      0.029123  


In [95]:
# Drop features where the means of the pos are similar (based on t-test)
import scipy.stats as stats
# Store t-values as a list
tvalues = []
for column in pos_counts:
    tval = stats.ttest_ind(pos_counts[pos_counts['text_source'] == 'Austen'], pos_counts[pos_counts['text_source'] == 'Carroll'])
    tvalues.append(tval)
print(tvalues)

C:\Users\maken\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: unsupported operand type(s) for +: 'spacy.tokens.span.Span' and 'spacy.tokens.span.Span'

In [162]:
d3 = pd.DataFrame()
d3['text_sentence'] = sentences[0]
d3['text_source'] = sentences[1]
d3.loc[:, word_counts] = 0

for i, sentence in enumerate(d3['text_sentence']):
    word_counts = 0
    words = [token
             for token in sentence
             if (
                 not token.is_punct
                 and not token.is_stop
                 
                 )]
    word_counts = len(words)
    d3.loc[i, word_counts] = word_counts
    if i % 500 == 0:
        print('Processing row {}'.format(i))
        
d3.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,text_sentence,text_source,1,20,23,8,4,35,5,17,...,46,49,67,51,42,40,69,76,59,48
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(So, she, was, considering, in, her, own, mind...",Carroll,0,NaN,23.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll,0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"((, when, she, thought, it, over, afterwards, ...",Carroll,0,NaN,NaN,NaN,NaN,35.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
